In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn import preprocessing
from collections import Counter

In [ ]:
df = pd.read_csv("Cleandata.csv")
df

,Unnamed: 0,Year,Severity,Start_Lat,Start_Lng,Distance(mi),Street,City,County,State,Airport_Code,Temperature(F),Wind_Chill(F),Visibility(mi),Wind_Direction,Weather_Condition,Traffic_Signal,Sunrise_Sunset,TimeDiff
0,0,2016,2,39.063148,-84.032608,0.010,State Route 32,Williamsburg,Clermont,OH,KI69,36.0,33.3,10.0,SW,Overcast,True,Night,30.0
1,1,2016,3,39.747753,-84.205582,0.010,I-75 S,Dayton,Montgomery,OH,KDAY,35.1,31.0,9.0,SW,Mostly Cloudy,False,Night,30.0
2,2,2016,2,39.627781,-84.188354,0.010,Miamisburg Centerville Rd,Dayton,Montgomery,OH,KMGY,36.0,33.3,6.0,SW,Mostly Cloudy,True,Day,30.0
3,3,2016,3,40.100590,-82.925194,0.010,Westerville Rd,Westerville,Franklin,OH,KCMH,37.9,35.5,7.0,SSW,Light Rain,False,Day,30.0
4,4,2016,2,39.758274,-84.230507,0.000,N Woodward Ave,Dayton,Montgomery,OH,KDAY,34.0,31.0,7.0,WSW,Overcast,False,Day,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314280,314280,2017,2,28.879630,-82.092960,0.722,I-75 S,Wildwood,Sumter,FL,KVVG,80.0,80.0,10.0,VAR,Light Rain with Thunder,False,Day,360.0
314281,314281,2017,2,37.993881,-92.795869,0.000,Sunny Slope Dr,Camdenton,Camden,MO,KH21,75.0,75.0,10.0,N,Cloudy,False,Day,360.0
314282,314282,2017,2,46.617235,-113.051251,0.000,US-12 W,Drummond,Granite,MT,K3DU,89.0,89.0,3.0,SSE,Haze,False,Day,360.0
314283,314283,2017,4,46.672100,-113.153690,9.486,US-12 W,Drummond,Granite,MT,K3DU,57.0,57.0,4.0,CALM,Haze,False,Night,360.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314285 entries, 0 to 314284
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Unnamed: 0         314285 non-null  int64  
 1   Year               314285 non-null  int64  
 2   Severity           314285 non-null  int64  
 3   Start_Lat          314285 non-null  float64
 4   Start_Lng          314285 non-null  float64
 5   Distance(mi)       314285 non-null  float64
 6   Street             314285 non-null  object 
 7   City               314285 non-null  object 
 8   County             314285 non-null  object 
 9   State              314285 non-null  object 
 10  Airport_Code       314285 non-null  object 
 11  Temperature(F)     314285 non-null  float64
 12  Wind_Chill(F)      314285 non-null  float64
 13  Visibility(mi)     314285 non-null  float64
 14  Wind_Direction     314285 non-null  object 
 15  Weather_Condition  314285 non-null  object 
 16  Tr

In [ ]:
df = df.drop(columns=['Unnamed: 0'])

KeyError: "['Unnamed: 0'] not found in axis"

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314285 entries, 0 to 314284
Data columns (total 18 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Year               314285 non-null  int64  
 1   Severity           314285 non-null  int64  
 2   Start_Lat          314285 non-null  float64
 3   Start_Lng          314285 non-null  float64
 4   Distance(mi)       314285 non-null  float64
 5   Street             314285 non-null  object 
 6   City               314285 non-null  object 
 7   County             314285 non-null  object 
 8   State              314285 non-null  object 
 9   Airport_Code       314285 non-null  object 
 10  Temperature(F)     314285 non-null  float64
 11  Wind_Chill(F)      314285 non-null  float64
 12  Visibility(mi)     314285 non-null  float64
 13  Wind_Direction     314285 non-null  object 
 14  Weather_Condition  314285 non-null  object 
 15  Traffic_Signal     314285 non-null  bool   
 16  Su

In [ ]:
features = ['Year', 'Severity', 'Start_Lat', 'Start_Lng', 'Distance(mi)', 'Temperature(F)', 'Wind_Chill(F)', 'Visibility(mi)', 'TimeDiff']
len(features)

9

In [ ]:
def IQR_method (df,n,features):
    """
    Takes a dataframe and returns an index list corresponding to the observations
    containing more than n outliers according to the Tukey IQR method.
    """
    outlier_list = []

    for column in features:
        # 1st quartile (25%)
        Q1 = np.percentile(df[column], 25)
        # 3rd quartile (75%)
        Q3 = np.percentile(df[column],75)

        # Interquartile range (IQR)
        IQR = Q3 - Q1

        # outlier step
        outlier_step = 1.5 * IQR

        # Determining a list of indices of outliers
        outlier_list_column = df[(df[column] < Q1 - outlier_step) | (df[column] > Q3 + outlier_step )].index

        # appending the list of outliers
        outlier_list.extend(outlier_list_column)

    # selecting observations containing more than x outliers
    outlier_list = Counter(outlier_list)
    multiple_outliers = list( k for k, v in outlier_list.items() if v > n )

    # Calculate the number of records below and above lower and above bound value respectively
    df1 = df[df[column] < Q1 - outlier_step]
    df2 = df[df[column] > Q3 + outlier_step]

    print('Total number of outliers is:', df1.shape[0]+df2.shape[0])

    return multiple_outliers

In [ ]:
# detecting outliers
Outliers_IQR = IQR_method(df,1,features)

# dropping outliers
df_out = df.drop(Outliers_IQR, axis = 0).reset_index(drop=True)

Total number of outliers is: 67582


In [ ]:
df_out

,Year,Severity,Start_Lat,Start_Lng,Distance(mi),Street,City,County,State,Airport_Code,Temperature(F),Wind_Chill(F),Visibility(mi),Wind_Direction,Weather_Condition,Traffic_Signal,Sunrise_Sunset,TimeDiff
0,2016,2,39.063148,-84.032608,0.010,State Route 32,Williamsburg,Clermont,OH,KI69,36.0,33.3,10.0,SW,Overcast,True,Night,30.0
1,2016,3,39.747753,-84.205582,0.010,I-75 S,Dayton,Montgomery,OH,KDAY,35.1,31.0,9.0,SW,Mostly Cloudy,False,Night,30.0
2,2016,2,39.627781,-84.188354,0.010,Miamisburg Centerville Rd,Dayton,Montgomery,OH,KMGY,36.0,33.3,6.0,SW,Mostly Cloudy,True,Day,30.0
3,2016,3,40.100590,-82.925194,0.010,Westerville Rd,Westerville,Franklin,OH,KCMH,37.9,35.5,7.0,SSW,Light Rain,False,Day,30.0
4,2016,2,39.758274,-84.230507,0.000,N Woodward Ave,Dayton,Montgomery,OH,KDAY,34.0,31.0,7.0,WSW,Overcast,False,Day,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243014,2017,2,38.765632,-111.353007,0.159,US-50 W,Richfield,Sevier,UT,KU24,57.0,57.0,10.0,SW,Partly Cloudy,False,Night,360.0
243015,2017,2,39.764370,-105.593050,0.321,I-70 E,Dumont,Clear Creek,CO,K0CO,46.4,37.2,10.0,West,Overcast,False,Day,360.0
243016,2017,4,39.919750,-75.576910,0.277,Wilmington Pike,West Chester,Chester,PA,KOQN,63.0,63.0,10.0,CALM,Fair,True,Night,360.0
243017,2017,4,36.034630,-79.054100,0.331,Old NC Highway 10,Chapel Hill,Orange,NC,KIGX,63.0,63.0,10.0,NNE,Mostly Cloudy,False,Night,360.0


In [ ]:
df_out.to_csv("IQR_data.csv")